In [1]:
import requests
from bs4 import BeautifulSoup 
import lxml
import csv
from datetime import datetime


In [2]:
# Taking URL for resulted search or filtering
# Accept input from user
URL_required = input("Enter the URL: ")
print("You entered: ", URL_required)



Enter the URL: https://www.amazon.ae/s?k=lenovo+laptop&i=computers&rh=n%3A12050245031%2Cp_n_feature_thirty_browse-bin%3A27387944031%2Cp_n_feature_fifteen_browse-bin%3A12065142031%7C12065143031%2Cp_89%3ALenovo%2Cp_n_feature_thirty-two_browse-bin%3A27399040031%2Cp_36%3A-500000%2Cp_n_feature_nineteen_browse-bin%3A12064945031&dc&crid=1WUSXO4SXR4RP&qid=1679089723&rnid=12064942031&sprefix=lenovo+laptop%2Caps%2C164&ref=sr_pg_1
You entered:  https://www.amazon.ae/s?k=lenovo+laptop&i=computers&rh=n%3A12050245031%2Cp_n_feature_thirty_browse-bin%3A27387944031%2Cp_n_feature_fifteen_browse-bin%3A12065142031%7C12065143031%2Cp_89%3ALenovo%2Cp_n_feature_thirty-two_browse-bin%3A27399040031%2Cp_36%3A-500000%2Cp_n_feature_nineteen_browse-bin%3A12064945031&dc&crid=1WUSXO4SXR4RP&qid=1679089723&rnid=12064942031&sprefix=lenovo+laptop%2Caps%2C164&ref=sr_pg_1


In [3]:
website_URL = "https://www.amazon.ae"

In [4]:
# Storing file name and file path in global variables to be used in multiple cells
next_page = ""
file_name = f"Products_Data_Scraped.csv"
file_path = "C:/Users/yusuf/Data Science/Projects/Web Scraping/Scraping-Amazon-Products-Details"
path = file_path+"/"+file_name

In [5]:
# Requesting the webpage and adding the date to it
page = requests.get(URL_required)
# Checking the response of the getting the page, if respose is 200; this means we got it fine
page

<Response [200]>

In [6]:
# Chech next page availability
if page.ok:
    print('Request successful')
else:
    print("There is no left pages to find more products")


Request successful


In [7]:
def scrape_product_details(products, products_list):
    """ To collect product details """
    
    i = 0 # Number of products tracer
    
    for product in products:  # Start scraping each product details
        product_description = product.find("div",{"class":"a-section a-spacing-none a-spacing-top-small s-title-instructions-style"}
                                              ).find_next("h2",{"class":"a-size-mini a-spacing-none a-color-base s-line-clamp-4"}
                                                         ).find_next("a").find_next("span").text.strip()

        product_price_initial = product.find("div",{"class":"a-row a-size-base a-color-base"}
                                  ).find_next("a", {"class":"a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}
                                             ).find_next("span", {"class":"a-price"}
                                                        ).find_next("span", {"class":"a-price-whole"}).text
        product_price = product_price_initial[:-1]

        product_rate_initial = product.find("div", {"class":"a-section a-spacing-none a-spacing-top-micro"}
                                             ).find_next("div", {"class":"a-row a-size-small"}
                                                        ).find_next("span"
                                                                   ).find_next("span", {"class":"a-declarative"}
                                                                              ).find_next("a"
                                                                                         ).find_next("i").text
        product_rate = product_rate_initial[:3]
        
        
        product_rating_counts_initial = product.find("div", {"class":"a-section a-spacing-none a-spacing-top-micro"}
                                             ).find_next("div", {"class":"a-row a-size-small"}
                                                        ).find_next("span", {"class":"a-declarative"}
                                                                              ).find_next("a"
                                                                                         ).find_next("span"
                                                                                                    ).find_next("span").text

        
        product_rating_counts = product_rating_counts_initial[1]
        
        product_link =  website_URL +product.find("div",{"class":"a-section a-spacing-base"}
                                        ).find_next("div"
                                                   ).find_next("div"
                                                              ).find_next("div"
                                                                         ).find_next("h2"
                                                                                    ).find_next("a"
                                                                                               )['href']
        # a-section a-spacing-small puis-padding-left-small puis-padding-right-small
        # a-section a-spacing-none a-spacing-top-small s-title-instructions-style
        # a-section a-spacing-base
        
        # Storing the values in dictionary
        product = {'Product_Description':product_description, 'Product_Price':product_price,
                 'Product_Rate':product_rate,"Product_Rating_Counts":product_rating_counts,
                   'Product_Link':product_link}
        
                # Appending the product data to product_details list
        products_list.append(product)
        
        
        i +=1 
        if i <= 1:  # To distinguish first cycle
            all_products_datails = products_list # First cycle for the loop
        else: 
            all_products_datails = all_products_datails + products_list # Later cycles
            
            

    return(all_products_datails)


In [8]:

def store_in_csv(final_products_details, file_name, path):
    """To store the scraped products data in csv file"""

    # Creating Keys to use in product dictionary
    keys = ['Product_Description', 'Product_Price', 'Product_Rate', 'Product_Rating_Counts', 'Product_Link']

    # Storing the championships details in csv file
    with open(path, 'w', newline="") as product_data:
        dict_writer = csv.DictWriter(product_data, fieldnames=keys) 
        dict_writer.writeheader()

        # Looping through championships to store its data
        for product in final_products_details: 
            dict_writer.writerow(product)

        # Checking the file creation is done
        print(f"Products data saved to file {file_name}")

In [9]:

def scraping(page):
    """The main function to do the steps of scraping"""
    
    products_list = []
    
    src = page.content
    soup =  BeautifulSoup(src, "lxml")

    # Each product in the entered data
    products = soup.find("div", {"class":"s-main-slot s-result-list s-search-results sg-row"}
                        ).find_all("div", {"class":"sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20"})    
    
    # Gets all products     
    final_products_details = scrape_product_details(products, products_list)
    
    
    
    # Calling the function that stores the data scraped in csv file
    store_in_csv(final_products_details, file_name, path)

#### Doing all the work from the main function

In [10]:
scraping(page)

Products data saved to file Products_Data_Scraped.csv


## Working on the resulted data

### Openning the csv file and show the data

In [11]:
#Importing Libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from six.moves import urllib
import os 

In [12]:
#function to read the data

def load_data(file_path, file_name):
    
    csv_path = os.path.join(file_path, file_name)
    
    return(pd.read_csv(csv_path))



### Loading the data

In [13]:
# Loading the data using the load_data function
data = load_data(file_path, file_name)

In [14]:
data

,Product_Description,Product_Price,Product_Rate,Product_Rating_Counts,Product_Link
0,2022 Latest Lenovo ThinkPad E15 Gen 4 Business...,"2,711",3.3,5,https://www.amazon.ae/Lenovo-ThinkPad-Business...
1,2022 Latest Lenovo ThinkPad E15 Gen 4 Business...,"3,143",2.6,7,https://www.amazon.ae/Lenovo-ThinkPad-Business...
2,2022 Latest Lenovo ThinkPad E15 Gen 4 Business...,"2,711",3.3,5,https://www.amazon.ae/Lenovo-ThinkPad-Business...
3,2022 Latest Lenovo ThinkPad E15 Gen 4 Business...,"3,143",2.6,7,https://www.amazon.ae/Lenovo-ThinkPad-Business...
4,2022 Latest Lenovo ThinkPad E15 Gen 4 Business...,"2,711",3.3,5,https://www.amazon.ae/Lenovo-ThinkPad-Business...
...,...,...,...,...,...
296,"2020 Lenovo ThinkPad E15 Business Laptop, 15.6...","3,290",4.1,4,https://www.amazon.ae/Lenovo-ThinkPad-i5-10210...
297,"Lenovo IdeaPad 3 15.6"" HD Touchscreen Laptop, ...","4,216",4.1,4,https://www.amazon.ae/Lenovo-IdeaPad-Touchscre...
298,2022 Latest Lenovo ThinkPad E15 Gen 2 Business...,"3,399",5.0,1,https://www.amazon.ae/Lenovo-ThinkPad-Business...
299,"2020 Latest Lenovo ThinkPad E15 Laptop 15.6"" F...","3,408",5.0,1,https://www.amazon.ae/Lenovo-ThinkPad-Display-...


## To be enhanced and built in rich details and more columns